## Import libraries

In [1]:
from network_optimizer import NetworkOptimizer
import synthetic_data_generators as sdg
import numpy as np
import pandas as pd

import time

## Import Synthetic Data

In [2]:
synth_network = pd.read_csv("../data/synth_network.csv")
synth_pool = pd.read_csv("../data/synth_pool.csv")
synth_members = pd.read_csv("../data/synth_members.csv")
synth_reqs = sdg.synth_reqs

In [3]:
print(f"Network size: {len(synth_network)}")
print(f"Pool size: {len(synth_pool)}")
print(f"Number of members: {len(synth_members)}")
synth_reqs

Network size: 100
Pool size: 100
Number of members: 1000


,specialty,county,provider_count,distance_req,min_access_pct,min_providers
0,cardiologist,wayne,1,15,90,5
1,pcp,wayne,2,15,90,10
2,ent,wayne,1,15,90,5
3,urologist,wayne,1,15,90,5
4,obgyn,wayne,1,15,90,5


## Run Optimization

### Create Network from scratch

In [5]:
optimizer = NetworkOptimizer(pool=synth_pool, 
                             members=synth_members,
                             adequacy_reqs=synth_reqs,
                             #no network
                             )

In [6]:
optimizer.optimize(10)

Optimization round 1 ...
Optimization round 2 ...
Optimization round 3 ...
Optimization round 4 ...
Optimization round 5 ...
Optimization round 6 ...
Optimization round 7 ...
Optimization round 8 ...
Optimization round 9 ...
Optimization round 10 ...
No more options for optimization
Average seconds per round of optimization: 0.8
Adequacy score for best network: 0.9736



In [7]:
optimizer.adequacy_detail

,specialty,county,provider_count,distance_req,min_access_pct,min_providers,total_members,members_with_access,servicing_providers,pct_with_access,adequacy_index
0,cardiologist,wayne,1,15,90,5,1000,1000,11,1.000,1.000
1,pcp,wayne,2,15,90,10,1000,990,10,0.990,0.990
2,ent,wayne,1,15,90,5,1000,1000,8,1.000,1.000
3,urologist,wayne,1,15,90,5,1000,910,7,0.910,0.910
4,obgyn,wayne,1,15,90,5,1000,968,6,0.968,0.968


In [8]:
optimizer.move_tracker

[('addition', 4),
 ('addition', 2),
 ('addition', 14),
 ('addition', 0),
 ('addition', 10),
 ('addition', 13),
 ('addition', 5),
 ('addition', 9),
 ('addition', 11)]

In [9]:
optimizer.performance_history

array([0.     , 0.25128, 0.4768 , 0.65592, 0.74724, 0.80772, 0.87188,
       0.94692, 0.9714 , 0.9736 ])

In [10]:
optimizer.best_network

,npi,specialty,group_id,efficiency,effectiveness,location_id,county,latitude,longitude,pct_serving
0,72,cardiologist,4,5,1,6,wayne,42.000698,-83.751425,0.334
1,56,urologist,4,1,1,7,wayne,42.199648,-83.761273,0.628
2,89,cardiologist,4,1,1,28,wayne,42.153219,-83.988858,0.323
3,27,ent,4,1,2,45,wayne,42.286955,-83.598814,0.587
4,81,cardiologist,4,4,1,46,wayne,42.296682,-83.854589,0.548
...,...,...,...,...,...,...,...,...,...,...
59,48,pcp,9,3,2,77,wayne,42.373220,-83.823846,0.480
60,93,cardiologist,9,1,3,92,wayne,42.223080,-83.734750,0.649
61,59,ent,11,4,4,42,wayne,42.354035,-83.533698,0.449
62,3,obgyn,11,4,4,58,wayne,41.936133,-83.595134,0.196


### Create Network from scratch using custom objective

In [17]:
def user_objective(self, network):
    if len(network) == 0:
        return 0
    else:
        return self.adequacy(network) * network.effectiveness.mean()

In [18]:
optimizer = NetworkOptimizer(pool=synth_pool, 
                             members=synth_members,
                             adequacy_reqs=synth_reqs,
                             # no network
                             user_objective=user_objective
                             )

In [19]:
optimizer.optimize(10)

Optimization round 1 ...
Optimization round 2 ...
Optimization round 3 ...
Optimization round 4 ...
Optimization round 5 ...
Optimization round 6 ...
Optimization round 7 ...
Optimization round 8 ...
Optimization round 9 ...
Optimization round 10 ...
No more options for optimization
Average seconds per round of optimization: 0.7
Adequacy score for best network: 0.9672000000000001



In [20]:
optimizer.adequacy_detail

,specialty,county,provider_count,distance_req,min_access_pct,min_providers,total_members,members_with_access,servicing_providers,pct_with_access,adequacy_index
0,cardiologist,wayne,1,15,90,5,1000,1000,8,1.000,1.000
1,pcp,wayne,2,15,90,10,1000,966,10,0.966,0.966
2,ent,wayne,1,15,90,5,1000,1000,8,1.000,1.000
3,urologist,wayne,1,15,90,5,1000,902,7,0.902,0.902
4,obgyn,wayne,1,15,90,5,1000,968,7,0.968,0.968


In [21]:
optimizer.move_tracker

[('addition', 2),
 ('addition', 1),
 ('addition', 14),
 ('addition', 0),
 ('addition', 5),
 ('addition', 13),
 ('addition', 11),
 ('addition', 10),
 ('addition', 6)]

In [22]:
optimizer.performance_history

array([0.        , 0.62345   , 1.25028471, 1.866864  , 2.31783568,
       2.54961143, 2.74885333, 2.94145333, 3.00906667, 3.03274576])

In [23]:
optimizer.best_network

,npi,specialty,group_id,efficiency,effectiveness,location_id,county,latitude,longitude,pct_serving
0,14,urologist,2,5,1,0,wayne,41.916604,-83.793571,0.175
1,5,urologist,2,2,5,4,wayne,42.189181,-83.655711,0.639
2,32,cardiologist,2,3,3,12,wayne,41.982906,-83.838536,0.268
3,62,urologist,2,2,2,22,wayne,42.113551,-83.472148,0.345
4,57,pcp,2,5,3,30,wayne,42.027369,-83.644040,0.359
5,5,urologist,2,2,5,39,wayne,42.361643,-83.532102,0.436
6,76,ent,2,5,5,63,wayne,42.396234,-83.950313,0.321
7,37,cardiologist,2,1,5,74,wayne,42.222695,-83.420171,0.356
8,37,cardiologist,2,1,5,78,wayne,42.061799,-83.649882,0.433
9,57,pcp,2,5,3,83,wayne,42.052663,-83.892604,0.324


In [24]:
optimizer.access_listing

,member_id,member_county,member_latitude,member_longitude,npi,specialty,group_id,efficiency,effectiveness,location_id,provider_county,provider_latitude,provider_longitude,distance,county,provider_count,distance_req,min_access_pct,min_providers
0,0,wayne,42.334931,-83.798905,14,urologist,2,5,1,0,wayne,41.916604,-83.793571,28.9,wayne,1,15,90,5
1,0,wayne,42.334931,-83.798905,47,ent,7,3,1,1,wayne,42.331189,-83.668563,6.7,wayne,1,15,90,5
2,0,wayne,42.334931,-83.798905,78,pcp,14,3,1,2,wayne,42.033343,-83.656534,22.1,wayne,2,15,90,10
3,0,wayne,42.334931,-83.798905,60,pcp,7,5,1,3,wayne,42.223686,-83.744325,8.2,wayne,2,15,90,10
4,0,wayne,42.334931,-83.798905,5,urologist,2,2,5,4,wayne,42.189181,-83.655711,12.5,wayne,1,15,90,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,999,wayne,42.497080,-83.564959,99,cardiologist,3,5,3,95,wayne,42.027225,-83.624442,32.6,wayne,1,15,90,5
99996,999,wayne,42.497080,-83.564959,67,ent,6,3,5,96,wayne,42.213445,-83.888217,25.6,wayne,1,15,90,5
99997,999,wayne,42.497080,-83.564959,97,obgyn,6,2,4,97,wayne,42.334651,-83.865587,19.0,wayne,1,15,90,5
99998,999,wayne,42.497080,-83.564959,77,ent,7,4,3,98,wayne,42.343960,-83.761458,14.6,wayne,1,15,90,5
